In [1]:
import pandas as pd
import numpy as np
import string

In [2]:
%time
products=pd.read_csv('amazon_baby_subset.csv')

Wall time: 0 ns


In [3]:
products.head()

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1


In [4]:
imp_words=pd.read_json('important_words.json')

In [5]:
important_words=imp_words[0].tolist()

In [6]:
products=products.fillna({'review':''})

In [7]:
def remove_punct(text):
    translator=str.maketrans({k:''for k in string.punctuation})
    return text.translate(translator)

In [8]:
%time
products['clean_review']=products.loc[:,'review'].apply(remove_punct)

Wall time: 0 ns


In [9]:
products.head()

,name,review,rating,sentiment,clean_review
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...


In [10]:
%time
for word in important_words:
    products[word]=products['clean_review'].apply(lambda text:text.split().count(word))

Wall time: 0 ns


In [11]:
products.head(2)

,name,review,rating,sentiment,clean_review,baby,one,great,love,use,...,seems,picture,completely,wish,buying,babies,won,tub,almost,either
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
products['contains_perfect']=products.loc[:,'perfect']>=1

In [13]:
np.sum(products['contains_perfect'])

2955

In [14]:
def get_data(dataframe,features,labels):
    dataframe['constant']=1
    features=['constant']+features
    feats_matrix=dataframe[features].values
    output=dataframe[labels].values
    return feats_matrix,output

In [15]:
%time
important_matrix,imp_output=get_data(products,important_words,'sentiment')

Wall time: 0 ns


In [16]:
important_matrix.shape

(53072, 194)

In [17]:
def predict_prob(feats_matrix,coefficients):
    prob=1/(1+np.exp(-np.dot(feats_matrix,coefficients)))
    return prob

In [18]:
def feats_deriv(feat,errors):
    deriv=np.dot(feat,errors)
    return deriv

In [19]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1 + np.exp(-scores)))
    return lp

In [20]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        predictions = predict_prob(feature_matrix,coefficients)
        indicator = (sentiment==+1)
        errors = indicator - predictions
        for j in range(len(coefficients)):
            derivative = feats_deriv(errors,feature_matrix[:,j])
            coefficients[j]+=derivative*step_size
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print ('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [21]:
initial=[0.0]*194
coefficients=logistic_regression(important_matrix,imp_output,initial,1e-7,301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [22]:
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        predictions = predict_prob(feature_matrix,coefficients)
        indicator = (sentiment==+1)
        errors = indicator - predictions
        for j in range(len(coefficients)): # loop over each coefficient
            derivative = feats_deriv(errors,feature_matrix[:,j])
            coefficients[j]  += step_size * derivative
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print ('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

In [23]:
coefficients=logistic_regression(important_matrix,imp_output,initial,1e-7,301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [24]:
np.sum(np.dot(important_matrix,coefficients)>0)
products['pred']=0

In [25]:
products['pred'][np.dot(important_matrix,coefficients)>0]=1
products['pred'][np.dot(important_matrix,coefficients)<=0]=-1

C:\Users\shuoyan001\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\shuoyan001\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
np.sum(products['pred']==products['sentiment'])

39903

In [28]:
coef=coefficients[1:]

In [35]:
pd.DataFrame(coef.tolist(),index=important_words).sort_values(0,ascending=True)

,0
would,-0.053860
product,-0.041511
money,-0.038982
work,-0.033070
even,-0.030051
disappointed,-0.028979
get,-0.028712
back,-0.027743
return,-0.026593
monitor,-0.024482
